# Motivation 

The purpose of this Notebook is to analyse the test output 

In [57]:
%load_ext autoreload
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import os
import sys

import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import matplotlib.pyplot as plt

import multiprocess as mp


# own libraries
sys.path.append('NetworkAnalysis/')

from NetworkAnalysis.ExperimentSet import ExperimentSet
from NetworkAnalysis import GraphHelper as gh
from NetworkAnalysis.utilities import clustering as cs
from NetworkAnalysis.utilities import sankey_consensus_plot as sky
from NetworkAnalysis.utilities.helpers import save_fig, survival_plot
from NetworkAnalysis.GraphToolExp import GraphToolExperiment as GtExp

pio.templates.default = "ggplot2"

pool = mp.Pool(mp.cpu_count())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
results_path = "../results/"
data_base = "../data/"
base_path = "../"
test_exps_path = "results/test/"
test_cltrs_path = "results/testCtrl/"
figures_path = "../test_results/"


mut_df = pd.read_csv(f"{data_base}/test_mutation_data.tsv",
                     sep="\t", index_col="gene")

# tf list
tf_path = f"{data_base}/TF_names_v_1.01.txt"
if os.path.exists(tf_path):
    tf_list = np.genfromtxt(fname=tf_path, delimiter="\t",
                            skip_header=1, dtype="str")

# Load experiments

In [59]:
exp_test = ExperimentSet("test", base_path=base_path, exp_path=test_exps_path,
                         mut_df=mut_df, sel_sets=None, exp_type="iNet")

exps = {}
# Compute some useful metadata for the experiments§
for idx, exp in enumerate(exp_test.get_exps()):
    if exp.sbm_method != "hsbm":  # or idx != 2:
        continue

    print(f"Loading Graph-Tool for {exp.type}")
    exps[exp.type] = GtExp.from_pgcna_inet(exp, rel_path="")
    exps[exp.type].export_to_gephi(save=False)

exp_test.exps = exps

##### Experiment labels:  dict_keys(['standard_5K_3TF_hsbm', 'standard_5K_4TF_hsbm'])
Loading Graph-Tool for standard_5K_3TF_hsbm
Loading Graph-Tool for standard_5K_4TF_hsbm


In [60]:
folders = next(os.walk(base_path + test_cltrs_path), (None, None, []))[1]
args, test_ctrls = [], {}
for folder in folders:
    hCtrl_path = f"{test_cltrs_path}/{folder}/"
    idx = int(folder.split("tctrl_")[-1])
    test_ctrls[idx] = ExperimentSet(
        "tCtrl", base_path, hCtrl_path, mut_df, sel_sets=None, rel_path="../", exp_type="iNet")
    test_ctrls[idx].export_to_gephi(save=False)

##### Experiment labels:  dict_keys(['standard_5K_3TF_hsbm', 'standard_5K_4TF_hsbm'])
##### Experiment labels:  dict_keys(['standard_5K_4TF_hsbm', 'standard_5K_3TF_hsbm'])
##### Experiment labels:  dict_keys(['standard_5K_3TF_hsbm', 'standard_5K_4TF_hsbm'])
##### Experiment labels:  dict_keys(['standard_5K_4TF_hsbm', 'standard_5K_3TF_hsbm'])
##### Experiment labels:  dict_keys(['standard_5K_3TF_hsbm', 'standard_5K_4TF_hsbm'])


# Selective edge pruning


* Work in progress * 